In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [11]:
pacientes = pd.read_csv(r'../Data/HC_Janeiro2021/HC_PACIENTES_1.csv', sep='|')

df = pd.read_csv(r'../Data/HC_Janeiro2021/HC_EXAMES_1.csv', sep='|')
df = pd.merge(df, pacientes)

df.insert(1, "EXAME", df['DE_EXAME']+" "+df['DE_ANALITO'])

out = df.pivot_table(index=['ID_aTENDIMENTO', 'IC_SEXO', 'AA_NASCIMENTO'], columns='EXAME', values='DE_RESULTADO', aggfunc='first')
out = out.reset_index(level=['IC_SEXO', 'AA_NASCIMENTO'])

print(out)

EXAME                            IC_SEXO AA_NASCIMENTO  \
ID_aTENDIMENTO                                           
005dc13e19b2523e0db4bf15156d6412       M          1969   
00759fffd8c3fcd6a790e67a8bbfcb59       M          1951   
00a495aa8cdc20aa8272e2ecab6a1c28       F          1941   
00b4a366c7da2ce894a4fa7a2b1c45bb       M          1956   
00b8650bb74e3c650560486ad30a5898       M          1942   
...                                  ...           ...   
ff71304ac140d9bfa8a60c8ae8542520       M          1976   
ff97502613f7d78596bb11f03a5d804d       F          1977   
ffcac17b109827d315f7e873266f677d       M          1948   
fffdf1bf42e05bd179b08b3c7ad7fd74       M          1976   
ffffe09e6250573cab8865ea566ea816       M          1959   

EXAME                            25-HIDROXIVITAMINA D 25-HIDROXIVITAMINA D  \
ID_aTENDIMENTO                                                               
005dc13e19b2523e0db4bf15156d6412                                       NaN   
00759fffd8c

In [9]:
convertedResult = pd.get_dummies(out, dummy_na=True)
print(convertedResult)

                                  25-HIDROXIVITAMINA D 25-HIDROXIVITAMINA D_10.0  \
ID_aTENDIMENTO                                                                     
005dc13e19b2523e0db4bf15156d6412                                           False   
00759fffd8c3fcd6a790e67a8bbfcb59                                           False   
00a495aa8cdc20aa8272e2ecab6a1c28                                           False   
00b4a366c7da2ce894a4fa7a2b1c45bb                                           False   
00b8650bb74e3c650560486ad30a5898                                           False   
...                                                                          ...   
ff71304ac140d9bfa8a60c8ae8542520                                           False   
ff97502613f7d78596bb11f03a5d804d                                           False   
ffcac17b109827d315f7e873266f677d                                           False   
fffdf1bf42e05bd179b08b3c7ad7fd74                                           F

In [4]:
valuesTable = []

for column in out.columns:
    uniqueValues = out[column].unique()
    notNullVal = uniqueValues[~pd.isnull(uniqueValues)]
    values = notNullVal.transpose()

    valuesTable.append(values)

stringValues = []

for values in valuesTable:
    for value in values:
        try:
            float(value)
        except:
            stringValues.append(value)

stringValues = pd.DataFrame(stringValues)[0].unique()
print(stringValues)

['< 4.0' 'Indetectavel' '*' ... '>20.0' '< 2.40' '<2.40']


In [ ]:
def has_digits(line):
    return any(char.isdigit() for char in line)

clearStringVal = []

for val in stringValues:
    if not has_digits(val):
        clearStringVal.append(val)

equivalenceMatrix = []
negativeCases = ["neg", "*", "não", "nao", "raro", "rara", "isolados", "inadequada", "zero", 
                 "inválido", "invalido", "ausência", "ausente", "ausencia", "failed", "indetectavel", "-",
                 "mínima", "minima", "discreta"]
singleCases = ["*", "r", "s", "i", "n", "w", "-"]

for val in clearStringVal:
    if len(val) > 1 and any(substring in val.lower() for substring in negativeCases):
        equivalenceMatrix.append((val, 0))
    elif len(val) == 1 and any(substring in val.lower() for substring in singleCases):
        equivalenceMatrix.append((val, 0))
    else:
        equivalenceMatrix.append((val, 1))

print(equivalenceMatrix)

[('Indetectavel', 0), ('*', 0), ('Negativa', 0), ('Reação citoquímica realizada para complementar diagnóstico.', 1), ('Foi  realizada a reação citoquímica.', 1), ('Presente', 1), ('Imediata', 1), ('NEGATIVO', 0), ('Realizado', 1), ('S', 0), ('R', 0), ('Não reagente', 0), ('Reagente', 1), ('negativo', 0), ('Positivo', 1), ('Negativo', 0), ('Duvidoso', 1), ('Não Reagente', 0), ('Ácido úrico', 1), ('.', 1), ('presente', 1), ('ausente', 1), ('Ausente', 1), ('Teste realizado através de curva de calibração híbrida para HBPM e HFN.', 1), ('Não detectado', 0), ('Detectado', 1), ('Inconclusivo', 1), ('Cocos Gram Positivos', 1), ('Não foram observados microrganismos', 0), ('Bacilos Gram Negativos', 0), ('Cocos Gram Positivos aos Pares e em Cadeias', 1), ('Leveduras', 1), ('Cocos Gram Positivos aos Pares', 1), ('Cocos Gram Positivos Agrupados', 1), ('Bacilos Gram Positivos', 1), ('Diplococos Gram Negativos', 0), ('Límpido e incolor', 1), ('Límpido e xantocrômico', 1), ('Levemente  hemorrágico', 1